## **Multiplicação de Matrizes:** Implementação _sequencial_, _Multithreading_ e _Multiprocessos_.

***

Este projeto visa observar as diferenças entre a implementação de algoritmos utilizando as abordagens sequencial, Multithreading e Multiprocessos, realizando testes de implementação e comparativos em termos de desempenho utilizando a linguagem **Python**, para a disciplina **Sistemas Operacionais** do curso de Bacharelado em Tecnologia da Informação, oferecido pela Universidade Federal do Rio Grande do Norte.

**Desenvolvido por:** José Manoel Freitas da Silva

***

### **1. Introdução**

A multiplicação de matrizes é uma operação fundamental em álgebra linear que desempenha um papel crucial em diversos campos da ciência da computação, engenharia e ciências exatas. Uma das principais vantagens da utilização desse método é a sua natureza altamente paralelizável, uma vez que multiplicação de matrizes é bastante suscetível à distribuição de tarefas, seja por meio de threads, em um ambiente multithread, ou por processos. Além disso, a multiplicação de matrizes pode ser escalada para grandes conjuntos de dados e dimensões, o que a torna relevante para problemas de alta complexidade encontrados em pesquisa científica e aplicações industriais.

Desse modo, iremos utilizar de uma de suas propriedades, chamada **independência de dados**, que nos permite calcular cada elemento da matriz resultante de forma independente dos demais dados da matriz, para implementar diversos conceitos da multiprogramação e analisar questões fundamentais de desempenho e otimização algorítmica, ressaltando a sua importância prática e sua capacidade de aproveitar ao máximo os recursos computacionais modernos. 

### **2. Metodologia**

Para esse projeto será feito 3 implementações, sendo elas:

* Uma implementação **Sequencial**, sem a utilização de multiprocessos ou multithreading;
* Uma implementação utilizando **Threads**;
* Uma implementação utilizando **Processos**.

Cada uma das implementações receberá duas matrizes **M1** e **M2** de tamanho **C**x**L** variados, com valores e dimensões randomizadas por meio de um algorítimo auxiliar implementado em **Python**. Para as implementações em multiprocessamento será utilizado ainda uma segunda variável **P**, que determina a quantidade de elementos que serão processados em cada thread ou processo criado, de modo a evitar problemas oriundos do excesso de processos abertos, frequentemente vistos em matrizes consideravelmente grandes.

Para quantificar o desempenho de cada implementação será medido o tempo de execução em cada uma das metodologias, realizando a plotagem de gráficos de desempenho com o intuito de visualizar o **tempo médio geral** e o **tempo médio em função de P**.

### **3. Considerações iniciais**

Nesse projeto será utilizado a linguagem de programação **Python**, aliada ao **Jupyter Notebook**, por permitir produzir gráficos e relatórios com facilidade e sem a necessidade de programas de terceiros, salvo apenas algumas bibliotecas do **Python**, como a _multiprocessing_, _threading_,_matplotlib_ e _numpy_.